# Bootes Brown photo-z cross

The photometric redshifts are calculated for the Brown aperture matched catalogue. Since in the manufacture of the masterlist we take the Brown aperture matched catalogue and add in some other obejcts. We here asign the Brown objects with their help id.

NOTE: There is some confusion as to whether the naming of the ID columns which may have caused misassociations. This is currently under investigation and the photo-z may be superseeded.

## 1. Load catalogues and join on IRAC ID

In [1]:
from astropy.table import Table, join, Column
import numpy as np

In [2]:
photo_z = Table.read('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public.fits')[
    'ID',
    'ID_OPTICAL', 
    'z1_median'
]
cross_ids = Table.read('../../dmu1/dmu1_ml_Bootes/data/'
                     + 'master_list_cross_ident_bootes_20190701.fits')[
    'help_id', 
    'brown_i_id'
]
#cross_ids['brown_i_id'].name = 'ID'


photo_z['ID_OPTICAL'].fill_value = -99

photo_z = photo_z.filled()
photo_z['brown_i_id'] = photo_z['ID_OPTICAL']  #?????
photo_z['brown_irac_id'] = photo_z['ID']       #????
photo_z = photo_z[photo_z['brown_i_id'] > -1]

In [3]:
new_cat = join(photo_z, cross_ids, join_type='left', keys='brown_i_id')

## 2. Replace bad photo-zs with nans

In [4]:
new_cat[new_cat['z1_median']<0]['z1_median'] = np.nan

In [5]:
new_cat['help_id'].fill_value = ''
new_cat = new_cat.filled()

## 3. Remove objects with duplicate help_ids

There seem to be a number of objects with duplicate ids which becaome duplicate help_ids. As far as I can tell these are totally identical so I remove the duplicates.

In [6]:
np.sum(new_cat['help_id'] == '')

409543

In [7]:
len(new_cat)

2067406

In [8]:
new_cat = new_cat[~(new_cat['help_id'] == '')]

In [9]:
len(new_cat)

1657863

In [10]:
new_cat['help_id'].dtype

dtype('S27')

In [11]:
len(np.unique(new_cat['help_id']))

1657470

In [12]:
new_cat.sort('help_id')
new_cat['help_id'][:-1][new_cat['help_id'][1:] == new_cat['help_id'][:-1]]

HELP_J142441.064+324758.688
HELP_J142441.765+322945.003
HELP_J142442.496+341145.082
HELP_J142443.618+350953.238
HELP_J142444.094+325217.503
HELP_J142446.166+343900.439
HELP_J142448.115+323206.565
HELP_J142451.431+340142.522
HELP_J142452.228+341546.019
HELP_J142454.562+324752.167
HELP_J142455.314+322844.779


In [20]:
new_cat[new_cat['help_id']=='HELP_J143841.165+335004.237']

ID,ID_OPTICAL,z1_median,brown_i_id,brown_irac_id,help_id
int64,int64,float64,int64,int64,bytes27
778163,778164,1.0382,778164,778163,HELP_J143841.165+335004.237


In [14]:
is_duplicate = new_cat['help_id'][1:] == new_cat['help_id'][:-1]
#np.append(is_duplicate,False)
new_cat = new_cat[~np.append(is_duplicate,False)]

In [21]:
new_cat[new_cat['help_id']=='HELP_J143841.165+335004.237']

ID,ID_OPTICAL,z1_median,brown_i_id,brown_irac_id,help_id
int64,int64,float64,int64,int64,bytes27
778163,778164,1.0382,778164,778163,HELP_J143841.165+335004.237


In [16]:
len(new_cat)

1657470

In [17]:
len(new_cat) == len(np.unique(new_cat['help_id']))

True

In [18]:
new_cat[:10].show_in_notebook()

idx,ID,ID_OPTICAL,z1_median,brown_i_id,brown_irac_id,help_id
0,1788579,1788580,0.9797,1788580,1788579,HELP_J142433.597+352308.103
1,1891098,1891099,0.8861,1891099,1891098,HELP_J142433.602+353326.121
2,1997887,1997888,0.7447,1997888,1997887,HELP_J142433.609+354416.174
3,1776605,1776606,1.8403,1776606,1776605,HELP_J142433.611+352156.195
4,1780408,1780409,0.3033,1780409,1780408,HELP_J142433.614+352219.215
5,2003638,2003639,1.7235,2003639,2003638,HELP_J142433.614+354452.719
6,1729998,1729999,0.3968,1729999,1729998,HELP_J142433.625+351736.189
7,1888109,1888110,2.416,1888110,1888109,HELP_J142433.629+353308.009
8,2000811,2000812,0.7871,2000812,2000811,HELP_J142433.635+354435.055
9,2029548,2029549,4.2342,2029549,2029548,HELP_J142433.641+354736.024


## 4. Write catalogue
This will be the catalogues that is used in dmu32 to merge photozs

In [19]:
new_cat.write('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public_with_help_id_20200923.fits', overwrite=True)